In [1]:
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from tqdm import tqdm # to create loadbard in for loop
import os.path
import json

# What will be achieved
This file will download the comments and their metada for each of the earning transcripts downloaded in the article folder.
The comments will be saved in the comments folder. If no comments for a given earning calls the file will not be created.

Note that normal get request, flagged the bot detection.
The code can be adapted to check, if we already have the comments so that it do not fetch them again. However, this would mean that new comments will not be downloaded.

The explication of the different api used are in a text file in the main folder

## First API data response

<pre>
dict(
    data : list( metadata for one comment),
    if no comments -> meta : don't care
    )
    
Metadata one comment:
dict(
    id : comment id (string),
    type : "comment",
    attributes : dict(
                      topParentId : comment id (int),
                      createdOn : date of creation (e.g. "2022-04-29T22:15:05.000-04:00"
                      pinned : don't know (bool)
                      )
    relationships : don't care
    links : don't care
                      )
)
</pre>


## Second API data reponse

<pre>
dict(
    data : list(metada for one comment),
    meta : don't care
)

Metadata for one comment:
dict(
    id : comment id (string),
    type : "comment",
    attributes : dict(
                      content : the comment (str),
                      edited : has it been edited afterward (bool),
                      parentId : don't know,
                      likesCount : (int),
                      rating : (int),
                      activeSubject : (bool),
                      createdOn : date of comment creation(str),
                      topParentId : don't know,
                      pinned : (bool)
                      )
    relationships : dict(
                         user : dict(
                                     data : dict(
                                                 id : user id (str),
                                                 type : e.g. "commentAuthor" (str)
                                                 )
                                     )
                         )
    links : don't care
)
</pre>



In [2]:
id_earning_calls = [name[:-5] for name in os.listdir("data/article/")]
# need to specify the right driver
executable_path = "./chromedriver_linux"
service = Service(executable_path)
option = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service,options=option)
nbr_dowloaded_comments_of_earning_call = 0
for id_earning_call in tqdm(id_earning_calls):
    url = "https://seekingalpha.com/api/v3/articles/"\
        +id_earning_call\
        +"/comment_maps?include=user&sort=-top_parent_id"
    driver.get(url)
    api_json_response_0 = driver.find_element(by=By.TAG_NAME, value = "body").text
    api_data_reponse_0 = json.loads(api_json_response_0)
    if api_data_reponse_0["data"]:
        with open("data/comment/"+ id_earning_call+".json", "w") as file:
            id_comments = [ data["id"] for data in api_data_reponse_0["data"]]
            url = "https://seekingalpha.com/api/v3/articles/"\
                +id_earning_call\
                +"/comments?comment_ids="\
                +",".join(id_comments)
            driver.get(url)
            api_json_response_1 = driver.find_element(by=By.TAG_NAME, value = "body").text
            api_data_response_1 = json.loads(api_json_response_1)
            file.write(str(api_data_response_1["data"]))
            nbr_dowloaded_comments_of_earning_call+=1
            
print("Number of downloaded comments of earning calls is : ",nbr_dowloaded_comments_of_earning_call)
if nbr_dowloaded_comments_of_earning_call!=0:
    # Check last downloaded comments, i.e. captcha/bot error 
    print(str(api_data_response_1["data"])[:200])

100%|███████████████████████████████████████████| 50/50 [00:26<00:00,  1.91it/s]

Number of downloaded comments of earning calls is :  1
[{'id': '92199410', 'type': 'comment', 'attributes': {'content': 'I’m getting messages of unsubscribing why I haven’t unsubscribe?', 'edited': False, 'parentId': None, 'likesCount': 0, 'rating': 0, 'a
